In [1]:
# Given the values of a single peakon on a coarse grid, this DNN finds the
# values of the peakon on a finer grid.
# Essentially, given the values shown by the @ symbols, it will output its 
# predictions for the values of the . symbols (and the four @ symbols bounding
# them).
#
#
# @   @   @   @   @   @
#
#
#
# @   @   @   @   @   @
#
#
#
# @   @   @...@   @   @
#         .....
#         .....
#         .....
# @   @   @...@   @   @
#
#
#
# @   @   @   @   @   @
#
#
#
# @   @   @   @   @   @

In [2]:
%run base/base_eq.ipynb
%run base/base_plot.ipynb
%run base/base_ml.ipynb

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = False

In [4]:
class BilinearInterp(FineGridPredictorMixin, BilinearInterpBase):
    pass


class PolyInterp(FineGridPredictorMixin, PolyInterpBase):
    pass

In [5]:
def gen_one_peakon_on_grid():
    """Returns a (feature, label) pair, where the features are the values of
    a single-peakon solution on a coarse grid, and the labels are the values of
    the single-peakon solution on a fine grid.
    """
    point, peakon = gen_one_peakon()
    return sol_on_grid(point, peakon)

In [15]:
### DNN hyperparameters
hidden_units = [2000] * 10
# i.e. the number of fine grid points.
logits = (fine_grid_fineness.t + 1) * (fine_grid_fineness.x + 1)
drop_rate = 0.4
drop_type = 'dropout'
preprocessor = ScaleDataOverall()
activation = tf.nn.relu
uuid = None

### Training hyperparameters #1
gradient_clip = 5.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 500  # How many steps to print the current loss.
model_dir_base = './saved_models/grid_single_peakon/'
model_dir = model_dir_str(model_dir_base, hidden_units, logits, 
                          drop_rate=drop_rate, drop_type=drop_type,
                          preprocessor=preprocessor, activation=activation, 
                          uuid=uuid)

dnn = create_dnn(hidden_units, logits, drop_rate=drop_rate,
                 drop_type=drop_type, model_dir=model_dir, 
                 log_steps=log_steps, activation=activation, 
                 gradient_clip=gradient_clip, batch_norm=False)
preprocessor.load(model_dir)

INFO:tensorflow:Using config: {'_model_dir': './saved_models/grid_single_peakon/2000x10_121_D04_ScaleDataOverall_relu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 500, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2cbd6c97b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Processor checkpoint file found, restoring values.


In [16]:
if TRAIN:
    ### Training hyperparameters #2
    batch_size = 30
    steps = 3000
    train_input_fn = preprocessor.data(BatchData(gen_one_data=gen_one_peakon_on_grid, batch_size=batch_size))
    dnn.train(input_fn=train_input_fn(use_tf=True), max_steps=steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])
    
    batch_size = 300
    steps = 6000
    train_input_fn = preprocessor.data(BatchData(gen_one_data=gen_one_peakon_on_grid, batch_size=batch_size))
    dnn.train(input_fn=train_input_fn(use_tf=True), max_steps=steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])

In [17]:
poly_deg = 10
results = plot_regressors([(Perfect(), None, 'Exact'),
                           (dnn, preprocessor.training(False), 'DNN'),
                           (BilinearInterp(), None, 'Bilinear'),
                           (PolyInterp(poly_deg), None, 'Poly-{}'.format(poly_deg))],
                          X_peak,
                          y_peak)
results[1].loss, results[2].loss, results[3].loss

FigureCanvasNbAgg()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/grid_single_peakon/2000x10_121_D04_ScaleDataOverall_relu/model.ckpt-5623
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


(38.39658922175019, 196.58519956744044, 107.3359233693232)

In [9]:
dnns_pros = create_dnns_pros_from_dir(model_dir_base)

INFO:tensorflow:Processor checkpoint file found, restoring values.
INFO:tensorflow:Using config: {'_model_dir': './saved_models/grid_single_peakon/1000x10_121_D04_ScaleDataByBatch_relu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f18ebf307f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Processor checkpoint file found, restoring values.
INFO:tensorflow:Using config: {'_model_dir': './saved_models/grid_single_peakon/1500x10_121_D04_ScaleDataByBatch_relu', '_tf_random_seed': None, '_save_summary_steps': 1

In [13]:
poly_deg = 10
average_reg = RegressorAverager(dnns_pros)
#average_reg.set_mask([False, False, True, True, True, True, True])
results = plot_regressors([(Perfect(), None, 'Exact'),
                           (average_reg, None, 'RegressorAverager'),
                           (BilinearInterp(), None, 'Bilinear'),
                           (PolyInterp(poly_deg), None, 'Poly-{}'.format(poly_deg))],
                          *gen_one_peakon_on_grid())
results[1].loss, results[2].loss, results[3].loss

FigureCanvasNbAgg()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/grid_single_peakon/1000x10_121_D04_ScaleDataByBatch_relu/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/grid_single_peakon/1500x10_121_D04_ScaleDataByBatch_relu/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/grid_single_peakon/2000x10_121_D04_ScaleDataOverall_relu/model.ckpt-5623
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


(7.356224928193971, 2.071188027877035, 4.0160781566844586)